### Deep Dream with Inception

In [1]:
from keras.applications import inception_v3
from keras import backend as K

Using TensorFlow backend.


In [3]:
#Loading pre-trained Inception
#disabling training operations
K.set_learning_phase(0)
model = inception_v3.InceptionV3(weights='imagenet', include_top=False)

In [4]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, None, None, 3 0                                            
__________________________________________________________________________________________________
conv2d_95 (Conv2D)              (None, None, None, 3 864         input_2[0][0]                    
__________________________________________________________________________________________________
batch_normalization_95 (BatchNo (None, None, None, 3 96          conv2d_95[0][0]                  
__________________________________________________________________________________________________
activation_95 (Activation)      (None, None, None, 3 0           batch_normalization_95[0][0]     
__________________________________________________________________________________________________
conv2d_96 

In [5]:
#dict mapping layernames to coefficients quantifying how much the layer's activation contributes to the loss
layer_contributions = {'mixed2':0.2, 'mixed3':3., 'mixed4':2., 'mixed5':1.5,}

In [6]:
#defining the loss to be maximized
#create a dcitionary that maps layernames to the layer instances
layer_dict = dict([(layer.name,layer) for layer in model.layers])
layer_dict

{'input_2': <keras.engine.input_layer.InputLayer at 0x1dd6754db38>,
 'conv2d_95': <keras.layers.convolutional.Conv2D at 0x1dd6754de10>,
 'batch_normalization_95': <keras.layers.normalization.BatchNormalization at 0x1dd6755f048>,
 'activation_95': <keras.layers.core.Activation at 0x1dd6755fd30>,
 'conv2d_96': <keras.layers.convolutional.Conv2D at 0x1dd67321d30>,
 'batch_normalization_96': <keras.layers.normalization.BatchNormalization at 0x1dd6721b940>,
 'activation_96': <keras.layers.core.Activation at 0x1dd6728dc50>,
 'conv2d_97': <keras.layers.convolutional.Conv2D at 0x1dd67424c88>,
 'batch_normalization_97': <keras.layers.normalization.BatchNormalization at 0x1dd6191eeb8>,
 'activation_97': <keras.layers.core.Activation at 0x1dd618ec1d0>,
 'max_pooling2d_5': <keras.layers.pooling.MaxPooling2D at 0x1dd6190d550>,
 'conv2d_98': <keras.layers.convolutional.Conv2D at 0x1dd61a225c0>,
 'batch_normalization_98': <keras.layers.normalization.BatchNormalization at 0x1dd619e4be0>,
 'activation_

In [7]:
#define loss by adding layer contributions to this scalar variable
loss = K.variable(0.)

for layer_name in layer_contributions:
    coeff = layer_contributions[layer_name]
    #retrieve the layer's output
    activation = layer_dict[layer_name].output
    
    scaling = K.prod(K.cast(K.shape(activation), 'float32'))
    
    #add L2 norm of the features of the layer to the loss
    loss += coeff * K.sum(K.square(activation[:, 2:-2, 2:-2, :])) / scaling

W1112 09:59:00.475097 27828 variables.py:2429] Variable += will be deprecated. Use variable.assign_add if you want assignment to the variable value or 'x = x + y' if you want a new python Tensor object.


In [8]:
#Gradient-ascent process

#tensor to hold generated image:the dream
dream = model.input

#compute the gradients of the dream w.r.t the loss
grads = K.gradients(loss, dream)[0]
#normalize the gradients
grads /= K.maximum(K.mean(K.abs(grads)), 1e-7)

grads

<tf.Tensor 'truediv_4:0' shape=(?, ?, ?, 3) dtype=float32>

In [9]:
#keras function to retrieve the value of the loss and gradients, given an input image
fetch_loss_and_grads = K.function([dream], [loss,grads])

In [10]:
def eval_loss_and_grads(x):
    outs = fetch_loss_and_grads([x])
    loss_value = outs[0]
    grad_values = outs[1]
    return loss_value,grad_values 

In [11]:
#function for gradient-ascent to run for number of iterations
def gradient_ascent(x, iterations, step, max_loss=None):
    for i in range(iterations):
        loss_value, grad_values = eval_loss_and_grads(x)
        if max_loss is not None and loss_value > max_loss:
            break
        print('loss value at',i,':',loss_value)
        x += step*grad_values
    return x

### Auxiliary functions 

In [16]:
import scipy
from keras.preprocessing import image

In [17]:
def resize_img(img,size):
    img = np.copy(img)
    factors = (1, float(size[0])/img.shape[1], float(size[1])/img.shape[2], 1)
    return scipy.ndimage.zoom(img,factors,order=1)

In [18]:
def save_img(img, fname):
    pil_img = deprocess_image(np.copy(img))
    scipy.misc.imsave(fname, pil_img)

In [23]:
def preprocess_image(image_path):
    img = image.load_img(image_path)
    img = image.img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = inception_v3.preprocess_input(img)
    return img

In [20]:
def deprocess_image(x):
    if K.image_data_format() == 'channels_first':
        x = x.reshape((3, x.shape[2], x.shape[3]))
        x = x.transpose((1,2,0))
    else:
        x = x.reshape((x.shape[1], x.shape[2], 3))
    x /= 2.
    x += 0.5
    x *= 255.
    x = np.clip(x, 0, 255).astype('uint8')
    return x    

In [34]:
#running gradient ascent over different successive scales
import numpy as np

step = 0.01
num_octave = 3
octave_scale = 1.4
iterations = 20
max_loss = 10.

base_image_path = r'C:\Users\v-nitbal\Documents\DL with Python - Francois Chollet\CNN for computer vision\scenery2.jpg'
img = preprocess_image(base_image_path)

In [35]:
#Prepare a list of shape tuples defining the different scales at which to run gradient ascent 
original_shape = img.shape[1:3]
successive_shapes = [original_shape]
for i in range(1, num_octave):
    shape = tuple([int(dim / (octave_scale ** i)) for dim in original_shape])
    successive_shapes.append(shape)
successive_shapes

[(457, 626), (326, 447), (233, 319)]

In [36]:
#ascending order of successive_shapes
successive_shapes = successive_shapes[::-1]
successive_shapes

[(233, 319), (326, 447), (457, 626)]

In [37]:
original_img = np.copy(img)
#resize to the smallest size
shrunk_original_img = resize_img(img, successive_shapes[0])

In [38]:
for shape in successive_shapes:
    print('Processing image shape',shape)
    #scales up the dream image
    img = resize_img(img, shape)
    
    #runs gradient ascent, altering the dream
    img = gradient_ascent(img, iterations=iterations, step=step, max_loss=max_loss)
    
    #scales up smaller version of the original image
    upscaled_shrunk_original_img = resize_img(shrunk_original_img, shape)
    
    #compute high quality version of the original image at this size
    same_size_original = resize_img(original_img, shape)
    
    #details lost when scaling up
    lost_detail = same_size_original - upscaled_shrunk_original_img
    
    #reinject lost details into the dream
    img += lost_detail
    
    shrunk_original_img = resize_img(original_img, shape)
    save_img(img, fname='dream_at_scale_'+str(shape)+'.png')
save_img(img, fname='final_dream.png')

Processing image shape (233, 319)
loss value at 0 : 1.7269647
loss value at 1 : 2.212258
loss value at 2 : 2.8975983
loss value at 3 : 3.522655
loss value at 4 : 4.2409678
loss value at 5 : 4.8925
loss value at 6 : 5.5404954
loss value at 7 : 6.130284
loss value at 8 : 6.7162495
loss value at 9 : 7.2278013
loss value at 10 : 7.8065653
loss value at 11 : 8.334682
loss value at 12 : 8.786233
loss value at 13 : 9.28951
loss value at 14 : 9.757768


C:\Users\v-nitbal\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: `imsave` is deprecated!
`imsave` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imwrite`` instead.
  This is separate from the ipykernel package so we can avoid doing imports until


Processing image shape (326, 447)
loss value at 0 : 2.6929464
loss value at 1 : 3.9596362
loss value at 2 : 5.04655
loss value at 3 : 6.0379796
loss value at 4 : 6.9275
loss value at 5 : 7.727541
loss value at 6 : 8.475445
loss value at 7 : 9.18998
loss value at 8 : 9.868133
Processing image shape (457, 626)
loss value at 0 : 2.8841429
loss value at 1 : 4.121398
loss value at 2 : 5.229141
loss value at 3 : 6.2501063
loss value at 4 : 7.230111
loss value at 5 : 8.243117
loss value at 6 : 9.309483
